In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
sys.path.append(os.path.dirname(os.path.abspath('')))

from abacus.splitter.params import SplitBuilderParams
from abacus.splitter.split_builder import SplitBuilder

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2

# Кейс №6. Выделение групп

In [2]:
df = pd.read_csv('./data/ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [3]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 20000,
        'target1': 30000
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    pvalue=0.05,
    n_top_cat=5,
    stat_test="ttest_ind",
    n_bins_rto = 6,
    bin_min_size = 500
)

In [4]:
split_builder = SplitBuilder(df, split_builder_params)

In [5]:
split = split_builder.collect()

In [6]:
split.head()

,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,numerator,denominator,conversion,strata,group_name
0,179.764218,177.979317,177.076199,181.064864,4.990621,2.444131,A,5,1,0.50121,2,3,1,120,target
1,177.623534,174.779951,170.028745,179.020354,4.238202,4.726904,B,13,4,0.49879,1,4,1,421,control
2,175.240488,170.410203,171.878569,178.750280,7.318058,5.226347,A,15,1,0.49879,2,1,1,11-1,target1
3,167.449021,163.500754,161.494712,169.214959,10.096040,6.560057,A,17,4,0.49879,2,4,1,40-1,target
4,186.789543,185.887011,183.315772,189.022250,10.740782,6.408251,A,29,2,0.50121,2,4,1,22-1,target
